In [44]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import math, random
import os, sys, time
import itertools
import cv2
import warnings
warnings.filterwarnings('ignore')

ImportError: dlopen(/anaconda3/envs/tfdeeplearning/lib/python3.5/site-packages/cv2.cpython-35m-darwin.so, 2): Library not loaded: @rpath/libfreetype.6.dylib
  Referenced from: /anaconda3/envs/tfdeeplearning/lib/libopencv_freetype.3.4.dylib
  Reason: Incompatible library version: libopencv_freetype.3.4.dylib requires version 23.0.0 or later, but libfreetype.6.dylib provides version 18.0.0

In [3]:
# Basic Parameters

max_epochs = 25
base_image_path =  "images/"
image_types = ["red", "green", "yellow"]
input_image_x = 32
input_image_y = 32
train_test_split_ratio = 0.9
batch_size = 32
checkpoint_name = "model.ckpt"

In [20]:
#Helper layer Funtion:

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, stride):
    return tf.nn.conv2d(x, W, strides=[1,stride,stride,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    

1. Our input neurons will be the shape of the image which is (32 x 32 x 3)
2. Because our data will be one-hot encoded, we have as many output neurons as we have classes

In [5]:
# Model

x = tf.placeholder(tf.float32, shape=[None, input_image_x, input_image_y, 3])
y_ = tf.placeholder(tf.float32, shape=[None, len(image_types)])

This is where we specify our first convolutional layers.

In [21]:
x_image = x

# Our first 3 conv layers, of 16 3X3 filters
W_conv1 = weight_variable([3, 3, 3, 16])
b_conv1 = bias_variable([16])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 1) + b_conv1)

W_conv2 = weight_variable([3, 3, 16, 16])
b_conv2 = bias_variable([16])
h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 1) + b_conv2)

W_conv3 = weight_variable([3, 3, 16, 16])
b_conv3 = bias_variable([16])
h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 1) + b_conv3)



In [22]:
# Our Pooling layer
h_pool4 = max_pool_2x2(h_conv3)

n1, n2, n3, n4 = h_pool4.get_shape().as_list()

W_fc1 = weight_variable([n2*n3*n4, 3])
b_fc1 = bias_variable([3])

# We flatten our pool layer into a fully connected layer

h_pool4_flat = tf.reshape(h_pool4, shape=[-1, n2*n3*n4])

y = tf.matmul(h_pool4_flat, W_fc1) + b_fc1

In [23]:
sess = tf.InteractiveSession()    

<l>*Our loss function is defined as computing softmax, and then cross entropy.*
<l>*We also specify our optimizer, which takes a learning rate, and a loss function.*
<l>*Finally, we initialize all of our variables which will tell us if our model is valid.*

In [28]:
# Our loss function and optimizer

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y))
train_step = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)
sess.run(tf.global_variables_initializer())

We need to load in our images. We do so using OpenCV's imread function. After loading in each image, we resize it to our input size.

With each loaded image, we also specify the expected output. For this, we use a one-hot encoding, creating an array of zeros represnting each class, and setting the index of the expected class number to 1

For example, if we have three classes, and we expect an order of: [red neuron, green neuron, yellow neuron] 
We initialize an array to [0, 0, 0] and if we loaded a yellow light, we change the last value to 1: [0, 0, 1] 
Finally, we shuffle our dataset. (It's generally useful to seed our random generator with 0 at the start of the program)

In [ ]:
saver = tf.train.saver()